In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image data/Boss.bmp
img = cv2.imread('data/Boss.bmp')

#convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#using adative thresholding seperate the black from the white in gray
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 8)

#create a function to remove small black pixels from the image 
def remove_small_black_pixels(img, size):
    #find the contours of the image
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #create a copy of the image
    img_copy = img.copy()
    #for each contour
    for cnt in contours:
        #if the contour is smaller than the size, remove it
        if cv2.contourArea(cnt) < size:
            cv2.drawContours(img_copy, [cnt], 0, (0,0,0), -1)
    #return the image
    return img_copy
clean_Image = cv2.bitwise_not(remove_small_black_pixels(cv2.bitwise_not(thresh), 5)) #used for cropping

#get the contours of the image
edged = cv2.Canny(clean_Image, 30, 200) # without canny, contour is created around whole image
contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
result = clean_Image.copy()
boxes = []
for c in contours:
    (x, y, w, h) = cv2.boundingRect(c)
    boxes.append([x,y, x+w,y+h])

boxes = np.asarray(boxes)
left = np.min(boxes[:,0])
top = np.min(boxes[:,1])
right = np.max(boxes[:,2])
bottom = np.max(boxes[:,3])

#crops around the contour of the image
ROI = thresh[top:bottom, left:right].copy()
ROI_original = img[top:bottom, left:right].copy() # cropped image of the original image
#smooth the image
ROI_smooth = cv2.GaussianBlur(ROI_original, (5,5), 0)

ret, thresh = cv2.threshold(ROI, 180, 255, cv2.THRESH_BINARY)
ROI_original[thresh == 255] = 255

cv2.imshow('ROI', ROI_original)
cv2.waitKey(0)

#save the image to data/Boss_transparent.bmp
cv2.imwrite('data/Boss_transparent.png', ROI)





True